In [1]:
"""Generating an IBOR extract

Attributes
----------
cocoon - seed_data
holdings
"""

'Generating an IBOR extract\n\nAttributes\n----------\ncocoon - seed_data\nholdings\n'

# Generating an IBOR extract with LUSID's GetHoldings method

This notebooks shows how you can use the [GetHoldings](https://www.lusid.com/docs/api/#operation/GetHoldings) API to generate IBOR extracts.

## Setup LUSID

In [2]:
# Import system packages

import os

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling and data management packages
import pandas as pd
import numpy as np
import json
import openpyxl
pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

In [3]:
# Load a mapping file for formatting DataFrame columns

with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_txn_json_mappings = json.load(mappings_file)

## Load transactions into a new scope

For the purpose of this demo, we first need to make sure we have a portfolio with some data. In the code below, we create a new portfolio called <b>EQUITY_UK</b> with some transactions from the <i>equity_transactions.csv</i> file. The focus of this notebook is on getting holdings so we use the <i>seed_data()</i> function to quickly generate a demo portfolio. For a more in-depth look into loading transactions and an instrument master in LUSID, see our long-form [tutorial](https://support.finbourne.com/how-do-i-create-holdings) on the support page. 

In [4]:
# Create a new scope
# Declare a variable to hold our portfolio name

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [5]:
# Load a file of equity transactions

transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [6]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data_response = seed_data(api_factory,
          ["portfolios", "instruments", "transactions"],
          scope,
          transactions_file,
          "csv"
         )

## GetHoldings

We call the <b>GetHoldings</b> method to generate an IBOR extract. This method requires two inputs: a portfolio and a scope. If we do not pass a date, then the default position date is today. You can modify the <b>scope</b> and <b>code</b> below to generate holdings for your own portfolios.

In [7]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [8]:
# Call the get_holdings method on the TransactionPortfoliosApi object
# The python SDK uses snake case get_holdings to represent LUSID's GetHoldings method
# We also pass a property to show each instrument's name

holdings_response = txn_port_api.get_holdings(scope=scope,
                                     code=portfolio_code,
                                     property_keys=["Instrument/default/Name"])

In the holdings's response, we can see a seperate value (represented as a row in the DataFrame) for each holdings. Holdings in securities have a <b>HoldingType</b> of <b>P</b> while holdings of cash balances have a <b>HoldingType</b> of <b>B</b>. See the linked [tutorial](https://support.finbourne.com/how-do-i-create-holdings) for a full list of holding types in LUSID.

In [9]:
# Convert JSON response to DataFrame

lusid_response_to_data_frame(holdings_response, rename_properties=True).head(10)

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_YN3FEUD5,{},Aviva,EQUITY_UK,3967-b0e1-f73b-26,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7EQ664NC,{},BHP,EQUITY_UK,3967-b0e1-f73b-26,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_F87JY45S,{},Barclays,EQUITY_UK,3967-b0e1-f73b-26,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_8X5TTJTL,{},BP,EQUITY_UK,3967-b0e1-f73b-26,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_H9EN8GYB,{},HSBC,EQUITY_UK,3967-b0e1-f73b-26,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,CCY_GBP,{},GBP,EQUITY_UK,3967-b0e1-f73b-26,B,3260000.0,3260000.0,3260000.0,GBP,0.0,GBP
6,LUID_OSXJYVK6,{},Morrisons,EQUITY_UK,3967-b0e1-f73b-26,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
7,LUID_6RGYR2L4,{},Tesco,EQUITY_UK,3967-b0e1-f73b-26,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
8,LUID_2IXG0MUX,{},Rightmove,EQUITY_UK,3967-b0e1-f73b-26,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
9,LUID_OSHLMH9J,{},vodafone,EQUITY_UK,3967-b0e1-f73b-26,P,900000.0,900000.0,900000.0,GBP,0.0,GBP


## GetHoldings for a specific date and time

In the code below, we generate holdings again, but this time we provide an effective date of 5 January 20202 at 12:30 PM (UTC). As we can see in the DataFrame below, the portfolio has a different set of holdings on that date.

If you pass a date with no time (e.g. <b>2020-01-05</b> then LUSID will set the default time to midnight or <b>00:00:00</b>)

In [10]:

holdings_response_fifth_jan = txn_port_api.get_holdings(scope=scope,
                                     code=portfolio_code,
                                     effective_at="2020-01-05T12:30:00Z",
                                     property_keys=["Instrument/default/Name"])

In [11]:
# Convert JSON response to DataFrame

holdings_fifth_jan = lusid_response_to_data_frame(holdings_response_fifth_jan, rename_properties=True)

#Format the columns we want to view

column_rename = {
    "instrument_uid": "luid",
    "Name(default-Properties)": "instrumentName",
    "holding_type": "holdingType",
    "units": "units",
    "settled_units": "settledUnits",
    "cost.amount": "costAmount",
    "cost.currency": "costCurrency",
    "cost_portfolio_ccy.currency": "portfolioCurrency"
}

holdings_fifth_jan = holdings_fifth_jan.rename(columns = column_rename)[column_rename.values()].copy()
display(holdings_fifth_jan)

,luid,instrumentName,holdingType,units,settledUnits,costAmount,costCurrency,portfolioCurrency
0,LUID_YN3FEUD5,Aviva,P,132000.0,132000.0,660000.0,GBP,GBP
1,LUID_7EQ664NC,BHP,P,120000.0,120000.0,2160000.0,GBP,GBP
2,LUID_F87JY45S,Barclays,P,300000.0,300000.0,600000.0,GBP,GBP
3,LUID_8X5TTJTL,BP,P,200000.0,200000.0,1000000.0,GBP,GBP
4,LUID_H9EN8GYB,HSBC,P,40000.0,40000.0,240000.0,GBP,GBP
5,CCY_GBP,GBP,B,7340000.0,7340000.0,7340000.0,GBP,GBP


## What transactions produced my holdings?

The holdings in LUSID's <b>Transaction Portfolios</b> are produced as the result of transactions. You can link a holding back to its source transaction using the build transactions endpoint. Let's do a deep-dive into <i>Tesco</i> which is one of our holdings.

In [12]:
# Call the build transactions endpoint

build_transactions_response = txn_port_api.build_transactions(scope=scope,
                               code=portfolio_code,
                                property_keys=["Instrument/default/Name"],
                               transaction_query_parameters = models.TransactionQueryParameters(start_date="2020-01-01",
                                                                                   end_date="2020-12-31"))

By calling the build transactions endpoint, we can see that two trades produce our holding of 12,000 units:

* One transaction on the 17 January 2020 for 8,000 units
* Then a second transaction on the 18 January 2020 for 4,000 units

The <b>ResultantHolding</b> column tracks the total holding in the portfolio for each unique instrument as of the transaction effective date and time. 

In [13]:
# Filter for a list of columns we want to show in the notebook

columns = ["TransactionId",
"TransactionType",
"LusidInstrumentId",
"InstrumentName",
"SettlementDate",
"Price",
"Units",
"ResultantHolding"]

# Target the rows for Tesco only

transactions_df = lusid_response_to_data_frame(build_transactions_response, column_name_mapping=build_txn_json_mappings, rename_properties=True)
transactions_df = transactions_df[columns]
transactions_df

,TransactionId,TransactionType,LusidInstrumentId,InstrumentName,SettlementDate,Price,Units,ResultantHolding
0,trd_0001,Buy,LUID_YN3FEUD5,Aviva,2020-01-04 00:00:00+00:00,5.0,120000.0,120000.0
1,trd_0002,Buy,LUID_YN3FEUD5,Aviva,2020-01-04 00:00:00+00:00,5.0,12000.0,132000.0
2,trd_0003,Buy,LUID_7EQ664NC,BHP,2020-01-04 00:00:00+00:00,18.0,60000.0,60000.0
3,trd_0004,Buy,LUID_7EQ664NC,BHP,2020-01-04 00:00:00+00:00,18.0,60000.0,120000.0
4,trd_0005,Buy,LUID_F87JY45S,Barclays,2020-01-04 00:00:00+00:00,2.0,150000.0,150000.0
5,trd_0006,Buy,LUID_F87JY45S,Barclays,2020-01-04 00:00:00+00:00,2.0,150000.0,300000.0
6,trd_0007,Buy,LUID_8X5TTJTL,BP,2020-01-04 00:00:00+00:00,5.0,100000.0,100000.0
7,trd_0008,Buy,LUID_8X5TTJTL,BP,2020-01-04 00:00:00+00:00,5.0,100000.0,200000.0
8,trd_0009,Buy,LUID_H9EN8GYB,HSBC,2020-01-04 00:00:00+00:00,6.0,20000.0,20000.0
9,trd_0010,Buy,LUID_H9EN8GYB,HSBC,2020-01-04 00:00:00+00:00,6.0,20000.0,40000.0


## Exporting holdings into an Excel file

You can then use standard panda's functions to export the holdings into an Excel file (or various other formats required by downstream systems or users).

In [14]:
holdings_fifth_jan.to_excel("extracts/holdings_20200105.xlsx")